In [1]:
import os
import sys

os.chdir("..")
sys.path.append("..")

In [2]:
import yaml

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from statsmodels.tsa.seasonal import STL
from tqdm import tqdm

from src.utils.data_loading import load_features, load_score, load_test_data
from src.utils.features import decomps_and_features
from src.utils.transformations import manipulate_trend_component, manipulate_seasonal_determination

/anaconda/envs/whatif/lib/python3.8/site-packages/gluonts/json.py:45: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(


In [3]:
model_name = "nbeats_g"
datasets = ["electricity_nips", "traffic_nips", "m4_yearly", "m4_quarterly", "m4_monthly", "m4_weekly", "m4_daily", "m4_hourly"]

outliers = {"electricity_nips": [341, 2111, 1189],
            "traffic_nips": [3237, 6049, 2017],
            "m4_yearly": [21379, 9541, 21725],
            "m4_quarterly": [6994, 20436, 7675],
            "m4_monthly": [42375, 16951, 9931],
            "m4_weekly": [295, 75, 285],
            "m4_daily": [165, 139, 3051],
            "m4_hourly": [222, 348, 340]}

inliers = {"electricity_nips": [608, 2044, 557],
           "traffic_nips": [4320, 0, 3616],
           "m4_yearly": [6957, 11749, 5772],
           "m4_quarterly": [15590, 3862, 11207],
           "m4_monthly": [39879, 29735, 29960],
           "m4_weekly": [157, 291, 98],
           "m4_daily": [2153, 3295, 3245],
           "m4_hourly": [207, 120, 140]}

In [4]:
def plot_ts(ts, fname, save_dir):
    plt.plot(ts)
    plt.xticks([])
    plt.yticks([])
    plt.savefig(os.path.join(save_dir, f"{fname}.svg"))
    plt.clf()


def plot_instance_space(pca_data, indexes, fname, save_dir):
    fig, ax = plt.subplots()
    plt.scatter(test_pca_data[:, 0], test_pca_data[:, 1], s=5, alpha=0.5, color="C0")
    for idx in indexes:
        plt.scatter(test_pca_data[idx, 0], test_pca_data[idx, 1], s=5, alpha=1, color="C1")
    
    plt.savefig(os.path.join(save_dir, f"{fname}.svg"))
    plt.clf()
    
    # plot again and label the positions of outliers and inliers
    fig, ax = plt.subplots()
    plt.scatter(test_pca_data[:, 0], test_pca_data[:, 1], s=5, alpha=0.5, color="C0")
    for idx in indexes:
        plt.scatter(test_pca_data[idx, 0], test_pca_data[idx, 1], s=5, alpha=1, color="C1")
    
    labels = ["a", "b", "c", "d", "e", "f"]
    for idx, label in zip(indexes, labels):
        ax.annotate(label, (test_pca_data[idx, 0], test_pca_data[idx, 1]))
    
    plt.savefig(os.path.join(save_dir, f"{fname}_labeled.svg"))
    plt.clf()

In [5]:
for dataset in datasets:
    datadir = f"data/{dataset}"
    experiment_dir = f"experiments/{dataset}/{model_name}"
    save_dir = f"./figures/timeseries/{dataset}"
    if not os.path.isdir(save_dir):
        os.makedirs(save_dir, exist_ok=True)
    
    with open(os.path.join(experiment_dir, "config.yaml"), "r") as f:
        config = yaml.load(f, Loader=yaml.FullLoader)
        
    train_features = load_features(datadir, train=True)
    test_features = load_features(datadir, train=False)
    test_data = load_test_data(dataset, config["context_length"] + config["prediction_length"])
    
    scaler = StandardScaler()
    norm_train_features = scaler.fit_transform(train_features)
    norm_test_features = scaler.transform(test_features)

    pca = PCA(n_components=2)
    train_pca_data = pca.fit_transform(norm_train_features)
    test_pca_data = pca.transform(norm_test_features)
    
    outlier_idx = []
    for label, outlier in zip(["a", "b", "c"], outliers[dataset]):
        outlier_idx.append(outlier)
        plot_ts(test_data[outlier], f"{label}_{outlier}", save_dir)
    
    inlier_idx = []
    for label, inlier in zip(["d", "e", "f"], inliers[dataset]):
        inlier_idx.append(inlier)
        plot_ts(test_data[inlier], f"{label}_{inlier}", save_dir)
    
    indexes = [*outlier_idx, *inlier_idx]
    plot_instance_space(test_pca_data, indexes, "instance_space", save_dir)

  0%|          | 0/2590 [00:00<?, ?it/s]

Loading test data


  0%|          | 0/6741 [00:00<?, ?it/s]

Loading test data


  1%|          | 243/23000 [00:00<00:09, 2420.31it/s]

...

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

In [6]:
def create_plot(ts1, ts2, fname, savedir):
    plt.draw()
    fig, (ax1, ax2)= plt.subplots(1, 2, figsize=(20, 10))
    ax1.plot(ts1)
    ax1.set_xticks([])
    ax2.plot(ts2)
    ax2.set_xticks([])
    
    plt.savefig(os.path.join(save_dir, f"{fname}_comparison.svg"))
    plt.clf()

In [8]:
dataset_transforms = {"electricity_nips": ["seas_dec", "slope_inc"],
                      "traffic_nips": ["seas_dec", "slope_inc", "slope_dec"],
                      "m4_yearly": ["slope_inc", "slope_dec"],
                      "m4_quarterly": ["seas_inc", "str_dec"],
                      "m4_monthly": ["str_dec", "slope_dec"],
                      "m4_weekly": ["slope_inc", "slope_dec"],
                      "m4_daily": ["seas_inc", "slope_dec", "slope_inc"],
                      "m4_hourly": ["slope_inc", "slope_dec"]}

targets = {"electricity_nips": [1213, 2111],
           "traffic_nips": [5086, 2017],
           "m4_yearly": [9541, 21379],
           "m4_quarterly": [20436, 7675],
           "m4_monthly": [42375, 9931],
           "m4_weekly": [75, 285],
           "m4_daily": [165, 139],
           "m4_hourly": [348, 340, 186]}

source_transform = {"electricity_nips": {0: dict(f=1, h=0.01, k=2.1, m=0), 10: dict(f=1, h=0.2, k=0.3, m=1)},
                    "traffic_nips": {0: dict(f=0.4, h=1, k=0.19, m=0.15), 10: dict(f=3, h=5, k=0.7, m=-0.2)},
                    "m4_yearly": {0: dict(f=1, h=1, k=1, m=-0.85), 1: dict(f=1.04, h=0.23, k=1, m=-0.95)},
                    "m4_quarterly": {0: dict(f=0.04, h=1, k=2, m=0), 20537: dict(f=1, h=1, k=5, m=1)},
                    "m4_monthly": {1: dict(f=0.01, h=1, k=1, m=0), 0: dict(f=1, h=1, k=1, m=-0.2)},
                    "m4_weekly": {0: dict(f=1, h=0.4, k=1, m=-0.4), 1: dict(f=1, h=1, k=1, m=0.25)},
                    "m4_daily": {0: dict(f=10, h=1, k=1, m=1), 1: dict(f=0.04, h=0.2, k=0.63, m=0)},
                    "m4_hourly": {0: dict(f=1, h=0.1, k=1, m=-0.3), "0": dict(f=1, h=0.07, k=1, m=0.8), 178: dict(f=1, h=1, k=0.01, m=0)}}

datasets = ["m4_hourly"]

for dataset in datasets:
    datadir = f"data/{dataset}"
    experiment_dir = f"experiments/{dataset}/{model_name}"
    save_dir = f"./figures/timeseries/{dataset}"
    if not os.path.isdir(save_dir):
        os.makedirs(save_dir, exist_ok=True)
    
    with open(os.path.join(experiment_dir, "config.yaml"), "r") as f:
        config = yaml.load(f, Loader=yaml.FullLoader)
        
    train_features = load_features(datadir, train=True)
    test_features = load_features(datadir, train=False)
    test_data = load_test_data(dataset, config["context_length"] + config["prediction_length"])
    
    scaler = StandardScaler()
    norm_train_features = scaler.fit_transform(train_features)
    norm_test_features = scaler.transform(test_features)

    pca = PCA(n_components=2)
    train_pca_data = pca.fit_transform(norm_train_features)
    test_pca_data = pca.transform(norm_test_features)
    
    for target, source in zip(targets[dataset], source_transform[dataset]):
        target_ts = test_data[target]
        source_ts = test_data[int(source)]
        decomp = decomps_and_features([source_ts], config["sp"])[0][0]
        
        f = source_transform[dataset][source]["f"]
        h = source_transform[dataset][source]["h"]
        m = source_transform[dataset][source]["m"] / len(target_ts)
        k = source_transform[dataset][source]["k"]
        
        generated_ts_trend = manipulate_trend_component(decomp.trend, f=f, h=h, g=1, m=m)
        generated_ts_seas = manipulate_seasonal_determination(decomp.seasonal, k=k)
        
        generated_ts = generated_ts_trend + generated_ts_seas + decomp.resid
        create_plot(target_ts, generated_ts, f"{target}_{source}", save_dir)

 29%|██▊       | 118/414 [00:00<00:00, 1121.13it/s]

Loading test data


100%|██████████| 414/414 [00:00<00:00, 1223.72it/s]


<Figure size 432x288 with 0 Axes>

<Figure size 1440x720 with 0 Axes>

<Figure size 1440x720 with 0 Axes>

<Figure size 1440x720 with 0 Axes>